In [ ]:
# def link_google(base='/content/drive', path='/My Drive/Colab Notebooks/'):
#     from google.colab import drive; drive.mount(base)
#     import sys; sys.path.append(base + path)

# link_google()

# !pip cache purge
# !pip install --upgrade pandas
# !pip install arrow pymongo

In [ ]:
import sys; sys.dont_write_bytecode=1
import warnings
from typing import Literal as Lit
from numpy import nan
import numpy as np
import pandas as pd
import deps as _
import modules as m
import Common
import gc
# from importlib import reload; reload(_); reload(m)

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)
# env='google_colab'
env='local'
gc.enable()

In [ ]:
TICKERS     = _.TECH_7
TOP         = _.TOP_30
TF, SFX, SRC, UNTIL, GRAN, ANUM = _.PRESET('Y')
PRDS, SLGS  = _.aPrds(TF, [1,2,3,4,5,6]).tolist(), _.aSlug(TF, [1,2,3,4,5,6]).tolist()


LI_RNK_1 = Common.LI_RNK_1[TF]
LI_RNK_2 = Common.LI_RNK_2[TF]
TP_RNK_1 = Common.TP_RNK_1[TF]
TP_RNK_2 = Common.TP_RNK_2[TF]

# RNK_1:TP_RNK_1  = 'High Cor 1Y' 
# RNK_2:TP_RNK_2  = 'High Gap 1Y'
# IND_1, IND_2    = _.np_remove_prefix([RNK_1, RNK_2])
WGHTS           = [1,1]
NORM_WGHTS      = False
RCOMB           = 'High Art'


DOWNLOAD, COMPUTE =  True, True
FULL_CLOCK        =  False
SYNC:_.TP_SYNC    = 'SAVE'
DIRECTORY         = 'Storage/'
RESULTS           = 'Results/'
FORMAT            = 'parquet'

In [ ]:
# import datetime as dt
# from dateutil.relativedelta import relativedelta

# UNTIL = None
# START = dt.date.today() - relativedelta(days=59)

In [ ]:
UNTIL = 'max'
Src2 = m.GET_SOURCE(DOWNLOAD, TICKERS, TF, SRC, UNTIL, FULL_CLOCK, start=None)
Src2 = _.pd_sync_storage(SYNC, Src2, DIRECTORY, 'Src2', SFX, fmt=FORMAT, env=env)

In [6]:
Scrn = m.COMPUTE_DATA(COMPUTE, Src2, GRAN, PRDS, SLGS)
Scrn = _.pd_sync_storage(SYNC, Scrn, DIRECTORY, 'Scrn', SFX, fmt=FORMAT, env=env)

del Src2;  gc.collect()

0

In [7]:
# Top, Agg = m.RANKED_FOLIOS(Scrn, TF, TOP, GRAN, ANUM,  RCOMB, IND_1,IND_2,RNK_1,RNK_2,  WGHTS, NORM_WGHTS)

In [8]:
CAGR   = _.pd_Storage(RESULTS, 'CAGR',   f'_{SFX}',  env=env)
Return = _.pd_Storage(RESULTS, 'Return', f'_{SFX}',  env=env)


for i, (RNK_1, RNK_2) in enumerate(list(_.it.product(LI_RNK_1[1:], LI_RNK_2[16:]))):
    try:
        print(i, RNK_1, RNK_2)

        IND_1, IND_2 = _.np_remove_prefix([RNK_1, RNK_2])

        Top, Agg = m.RANKED_FOLIOS(Scrn, TF, TOP, GRAN, ANUM,  RCOMB, IND_1,IND_2,RNK_1,RNK_2,  WGHTS, NORM_WGHTS)

        XLabel = _.combine_label(WGHTS, [RNK_1, RNK_2])
        CAGR.add_column(  XLabel, Top['CAGR']   .round(2))
        Return.add_column(XLabel, Top['Return'] .round(2))

        del Top, Agg;  gc.collect()
    except Exception as Error:
        print('iLoop:',i, 'Error:',Error)
pass

0 High Cor 1Y High Gap
iLoop: 0 Error: Columns must be same length as key
1 High Cor 1Y Low Gap
iLoop: 1 Error: Columns must be same length as key
2 High Cor 1Y High Gap 1Y
3 High Cor 1Y Low Gap 1Y
4 High Cor 1Y High Gap 3Y
5 High Cor 1Y Low Gap 3Y
6 High Cor 1Y High Gap 5Y
7 High Cor 1Y Low Gap 5Y
8 High Cor 1Y High Gap 10Y
9 High Cor 1Y Low Gap 10Y
10 High Cor 1Y High Gap 20Y
11 High Cor 1Y Low Gap 20Y
12 High Cor 1Y High Gap 40Y


KeyboardInterrupt: 

In [ ]:
# CAGR.load().to_excel(f'{RESULTS}CAGR.xlsx', SFX, index=False)
CAGR.load()

In [ ]:
# Return.load().to_excel(f'{RESULTS}Return.xlsx', SFX, index=False)
Return.load()

In [ ]:
# CAGR.create_new_empty({ 'Top':TOP })
# Return.create_new_empty({ 'Top':TOP })